# Visualise the ``Data`` used in this project

In [ ]:
import datetime
import logging
import os
import sys
import traceback

from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import yaml
from shapely.geometry import Point

from sdm_eurec4a import RepositoryPath, conversions, get_git_revision_hash
from sdm_eurec4a.identifications import (
    match_clouds_and_cloudcomposite,
    match_clouds_and_dropsondes,
    select_individual_cloud_by_id,
)

from sdm_eurec4a.reductions import shape_dim_as_dataarray
from sdm_eurec4a.visulization import (
    adjust_lightness_array,
    handler_map_alpha,
    set_custom_rcParams,
)

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
dark_colors = adjust_lightness_array(default_colors, amount=0.5)

repo_path = RepositoryPath("levante")()
print(repo_path)

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = Path("/home/m/m301096/repositories/sdm-eurec4a/notebooks/presentation")
print(script_dir)


fig_path = repo_path / "results" / script_dir.relative_to(repo_path)
print(fig_path)

fig_path.mkdir(parents=True, exist_ok=True)


def save_fig(
    fig,
    file_name: str,
    fig_dir_path: Path = fig_path,
    save: bool = True,
    format: list = [".svg", ".png"],
):
    """
    Save figures for given filename in figure directory paths for different formats.

    Parameters
    ----------
    """

    file_paths = [Path(fig_dir_path) / file_name + ending for ending in format]

    for fp in file_paths:
        fig.savefig(fp)

/home/m/m301096/repositories/sdm-eurec4a
/home/m/m301096/repositories/sdm-eurec4a/notebooks/presentation
/home/m/m301096/repositories/sdm-eurec4a/results/notebooks/presentation


In [ ]:
identified_clouds_path = repo_path / Path(
    "data/observation/cloud_composite/processed/identified_clouds/identified_clusters_rain_mask_5.nc"
)
cloud_composite_path = repo_path / Path(
    f"data/observation/cloud_composite/processed/cloud_composite_si_units.nc"
)
drop_sondes_path = repo_path / Path("data/observation/dropsonde/processed/drop_sondes.nc")
distance_path = repo_path / Path(
    "data/observation/combined/distance_relations/distance_dropsondes_identified_clusters_rain_mask_5.nc"
)
satellite_path = repo_path / Path(
    "data/observation/satellite/test/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20200261630156_BARBADOS-2KM-FD.level2.nc"
)

In [ ]:
cloud_composite["radius"]

In [ ]:
# %%

distance_IC_DS = xr.open_dataset(distance_path)

cloud_composite = xr.open_dataset(cloud_composite_path)

drop_sondes = xr.open_dataset(drop_sondes_path)

drop_sondes = drop_sondes.where(drop_sondes.alt <= 1600, drop=True)

# add relative humidity to the dataset

drop_sondes["relative_humidity"] = conversions.relative_humidity_from_tps(
    temperature=drop_sondes["air_temperature"],
    pressure=drop_sondes["pressure"],
    specific_humidity=drop_sondes["specific_humidity"],
)


# Add radii and altitude arrays to datasets

cloud_composite["radius_array"] = shape_dim_as_dataarray(
    da=cloud_composite["particle_size_distribution"], output_dim="radius"
)
cloud_composite["time_array"] = shape_dim_as_dataarray(
    da=cloud_composite["particle_size_distribution"], output_dim="time"
)
drop_sondes["alt_array"] = shape_dim_as_dataarray(da=drop_sondes["air_temperature"], output_dim="alt")

identified_clouds = xr.open_dataset(identified_clouds_path)
# select only clouds which are
# below 1500m and have a duration of at least 10s.
identified_clouds = identified_clouds.where(
    (identified_clouds["alt"] <= 1200)
    & (identified_clouds["alt"] >= 500)
    & (identified_clouds["vertical_extent"] <= 150)
    & (identified_clouds["duration"] >= np.timedelta64(3, "s"))
    # & (identified_clouds["liquid_water_content"] / identified_clouds["duration"].dt.seconds >=0.1)
    ,
    drop=True,
)

identified_clouds = xr.open_dataset(identified_clouds_path)
# select only clouds which are
# below 1500m and have a duration of at least 10s.
identified_clouds = identified_clouds.where(
    (identified_clouds["alt"] <= 1200)
    & (identified_clouds["alt"] >= 500)
    & (identified_clouds["vertical_extent"] <= 150)
    & (identified_clouds["duration"] >= np.timedelta64(3, "s"))
    # & (identified_clouds["liquid_water_content"] / identified_clouds["duration"].dt.seconds >=0.1)
    ,
    drop=True,
)

rename dataarray
rename dataarray
rename dataarray


In [ ]:
cc_cloud_base_time = cloud_composite.time.where(
    (cloud_composite["alt"] > 500) & (cloud_composite["alt"] <= 1200), drop=True
)
cc_cloud_base = cloud_composite.sel(time=cc_cloud_base_time)

In [ ]:
sonde_ids = drop_sondes.sonde_id.values

halo_ids = sonde_ids[["HALO" in v for v in sonde_ids]]
p3_ids = sonde_ids[["P3" in v for v in sonde_ids]]
# make sure no flight is used twice
assert sorted(np.concatenate([halo_ids, p3_ids])) == sorted(sonde_ids)

halo_launches = drop_sondes.time.where(drop_sondes.sonde_id.isin(halo_ids), drop=True)
p3_launches = drop_sondes.time.where(drop_sondes.sonde_id.isin(p3_ids), drop=True)

In [ ]:
cc_match = match_clouds_and_cloudcomposite(identified_clouds, cloud_composite)

In [ ]:
max_spatial_distance = 100
max_temporal_distance = np.timedelta64(3, "h")
ds_match = match_clouds_and_dropsondes(
    ds_clouds=identified_clouds,
    ds_sonde=drop_sondes,
    ds_distance=distance_IC_DS,
    max_spatial_distance=max_spatial_distance,
    max_temporal_distance=max_temporal_distance,
)

In [ ]:
sat = xr.open_dataset(satellite_path)

X = sat["longitude"]
Y = sat["latitude"]
mask = np.isfinite(Y) & np.isfinite(X)

sat = sat.where(mask, drop=True)

In [ ]:
# Draw a map of all dropsondes released during the campaign
import cartopy.crs as ccrs
import cartopy.feature as cfeature


def plot_map(
    fig,
    ax,
    cloud_composite: xr.Dataset,
    drop_sondes: xr.Dataset,
    identified_clouds: xr.Dataset = None,
    extent: list = [-60, -56, 12, 15],
):
    """ """

    ax.coastlines()
    ax.add_feature(cfeature.LAND)
    # ax.add_feature(cfeature.OCEAN, color="navy", alpha=0.2)
    # ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.gridlines(draw_labels=True, linestyle="--", alpha=0.5, color=[0.5, 0.5, 0.5])
    ax.set_extent(extent)

    ax.scatter(
        cloud_composite["lon"],
        cloud_composite["lat"],
        transform=ccrs.PlateCarree(),
        s=1,
        c=default_colors[0],
        marker=".",
        alpha=0.7,
        label="ATR",
        zorder=10,
    )

    ax.scatter(
        drop_sondes["flight_lon"],
        drop_sondes["flight_lat"],
        transform=ccrs.PlateCarree(),
        label="Drop sondes",
        color=dark_colors[1],
        marker="x",
        zorder=10,
        alpha=0.7,
    )

    if identified_clouds != None:
        ax.scatter(
            identified_clouds["lon"],
            identified_clouds["lat"],
            s=identified_clouds["horizontal_extent"] / 100,
            c="red",
            marker="o",
            alpha=0.5,
            # cmap = "Yellows_r",
            label="clouds",
            zorder=10,
        )


def plot_psd_pcolormesh_splitted(fig, ax, ds_cloudcomposite, split_radius: float = 4.5e-5):
    ds_lower = ds_cloudcomposite.sel(radius=slice(None, split_radius))
    ds_rain = ds_cloudcomposite.sel(radius=slice(split_radius, None))

    # plot cloud PSD distribution as pcolormesh

    vmin, vmax = 1, 1e8

    pc = ax.pcolormesh(
        ds_lower["radius"],
        ds_lower["time"],
        ds_lower["particle_size_distribution"].T,
        norm=mpl.colors.LogNorm(vmin, vmax),
        cmap="Blues",
        shading="nearest",
    )
    fig.colorbar(pc, ax=ax, label="Cloud and Drizzle drops #m$^{-3}$")
    pc = ax.pcolormesh(
        ds_rain["radius"],
        ds_rain["time"],
        ds_rain["particle_size_distribution"].T,
        norm=mpl.colors.LogNorm(vmin, 1e4),
        cmap="Reds",
        shading="nearest",
    )
    fig.colorbar(pc, ax=ax, label="Rain drops in #m$^{-3}$")
    ax.set_xscale("log")
    ax.set_xlabel("Radius in m")
    ax.set_ylabel("Time")


def plot_psd_scatter(fig, ax, ds_cloudcomposite, split_radius=4.5e-5):
    ds_lower = ds_cloudcomposite.sel(radius=slice(None, split_radius))
    ds_rain = ds_cloudcomposite.sel(radius=slice(split_radius, None))

    style = dict(
        marker="o",
        edgecolor="none",
        s=10,
        alpha=0.5,
    )

    ax.set_ylim(1e-1, 1e8)
    ax.set_yscale("log")
    ax.set_xscale("log")

    ax.scatter(
        ds_rain.radius_array,
        ds_rain.particle_size_distribution,
        c=default_colors[0],
        label="rain drops",
        **style,
    )

    ax.scatter(
        ds_lower.radius_array,
        ds_lower.particle_size_distribution,
        c=default_colors[1],
        label="cloud and drizzle",
        **style,
    )

    ax.legend(handler_map=handler_map_alpha(), loc="lower right")

    ax.set_xlabel("radius in m")
    ax.set_ylabel("Occurence in #/m3")


def plot_msd_scatter(fig, ax, ds_cloudcomposite, split_radius=4.5e-5):
    ds_lower = ds_cloudcomposite.sel(radius=slice(None, split_radius))
    ds_rain = ds_cloudcomposite.sel(radius=slice(split_radius, None))

    style = dict(
        marker="o",
        edgecolor="none",
        s=10,
        alpha=0.5,
    )

    ax.set_ylim(1e-10, 2e-4)
    ax.set_yscale("log")
    ax.set_xscale("log")

    ax.scatter(
        ds_rain.radius_array,
        ds_rain.mass_size_distribution,
        c=default_colors[0],
        label="rain drops",
        **style,
    )

    ax.scatter(
        ds_lower.radius_array,
        ds_lower.mass_size_distribution,
        c=default_colors[1],
        label="cloud and drizzle",
        **style,
    )

    ax.legend(handler_map=handler_map_alpha(), loc="lower right")

    ax.set_xlabel("radius in m")
    ax.set_ylabel("Mass in kg/m3")


def plot_thermodynamics(fig, ds_dropsondes, ds_cloud):
    axs = fig.subplots(ncols=2, nrows=2, sharey=True)
    scatter_style = dict(alpha=0.75, linewidth=0.7)

    ax_q, ax_ta = axs[0]
    ax_rh, ax_theta = axs[1]
    ax_q.set_xlim(0, 20)
    ax_rh.set_xlim(30, 110)
    ax_ta.set_xlim(285, 305)
    ax_theta.set_xlim(295, 308)

    # First plot
    ax_q.plot(
        1e3 * ds_dropsondes["specific_humidity"].T,
        ds_dropsondes["alt"].T,
        color=default_colors[0],
        # label="Spec. Hum.",
        **scatter_style,
    )

    ax_q.set_xlabel("Specific humidity [g/kg]")
    ax_q.set_ylabel("alt [m]")
    ax_q.xaxis.label.set_color(default_colors[0])  # Set the color of x-axis label
    ax_q.tick_params(axis="x", colors=default_colors[0])  # Set the color of x-axis ticks

    # First plot
    ax_rh.plot(
        ds_dropsondes["relative_humidity"].T,
        ds_dropsondes["alt"].T,
        color=default_colors[3],
        # label="Spec. Hum.",
        **scatter_style,
    )
    ax_rh.axvline(100, color="grey", linestyle="-", label="Saturation")
    ax_rh.set_xlabel("Relative humidity [%]")
    ax_rh.set_ylabel("alt [m]")
    ax_rh.xaxis.label.set_color(default_colors[3])  # Set the color of x-axis label
    ax_rh.tick_params(axis="x", colors=default_colors[3])  # Set the color of x-axis ticks

    # Second plot on a new x-axis
    ax_ta.plot(
        ds_dropsondes["air_temperature"].T,
        ds_dropsondes["alt"].T,
        color=default_colors[1],
        # label="Air Temperature",
        **scatter_style,
    )

    ax_ta.set_xlabel("Air Temperature [K]")
    ax_ta.xaxis.label.set_color(default_colors[1])  # Set the color of x-axis label
    ax_ta.tick_params(axis="x", colors=default_colors[1])  # Set the color of x-axis ticks

    # Thrid plot on a new x-axis
    ax_theta.plot(
        ds_dropsondes["potential_temperature"].T,
        ds_dropsondes["alt"].T,
        color=default_colors[2],
        # label="Pot. Temperature",
        **scatter_style,
    )

    ax_theta.set_xlabel("Pot. Temperature [K]")
    ax_theta.xaxis.label.set_color(default_colors[2])  # Set the color of x-axis label
    ax_theta.tick_params(axis="x", colors=default_colors[2])  # Set the color of x-axis ticks

    for ax in axs.flatten():
        ax.set_ylim(0, 1500)
        ax.axhline(ds_cloud.alt, color="r", linestyle="--", label="PSD measurement")
        ax.legend(handler_map=handler_map_alpha())
        ax.tick_params(axis="x", labelrotation=-33)

In [ ]:
individual_cloud = select_individual_cloud_by_id(identified_clouds, 37)
cc_individual = match_clouds_and_cloudcomposite(individual_cloud, cloud_composite)
max_spatial_distance = 100
max_temporal_distance = np.timedelta64(3, "h")
ds_individual = match_clouds_and_dropsondes(
    ds_clouds=individual_cloud,
    ds_sonde=drop_sondes,
    ds_distance=distance_IC_DS,
    max_spatial_distance=max_spatial_distance,
    max_temporal_distance=max_temporal_distance,
)
individual_cloud.time

In [ ]:
fig, axs = plt.subplots(
    ncols=2, figsize=(20, 10), layout="constrained", subplot_kw={"projection": ccrs.PlateCarree()}
)


plot_map(fig, axs[0], cc_cloud_base, drop_sondes.sel(time=halo_launches))
plot_map(fig, axs[1], cc_match, ds_match, identified_clouds)


lgnd = axs[0].legend()
for handle in lgnd.legend_handles:
    handle.set_sizes([10.0])

axs[0].set_title("Dropsonde and ART locations overview")
# fig.savefig(figure_dir / "sonde_art_locations_overview.png", dpi=300, transparent=False)

Text(0.5, 1.0, 'Dropsonde and ART locations overview')

In [ ]:
fig, ax = plt.subplots(
    nrows=1, figsize=(5, 5), layout="constrained", subplot_kw={"projection": ccrs.PlateCarree()}
)


plot_map(fig, ax, cc_individual, ds_individual, individual_cloud)

ax.pcolormesh(
    sat["longitude"],
    sat["latitude"],
    sat["refl_0_47um_nom"],
    cmap="Greys_r",
    zorder=1,
    vmin=sat["refl_0_47um_nom"].min()
    - 0.5 * (sat["refl_0_47um_nom"].max() - sat["refl_0_47um_nom"].min()),
    vmax=sat["refl_0_47um_nom"].max()
    - 0.5 * (sat["refl_0_47um_nom"].max() - sat["refl_0_47um_nom"].min()),
)

ax.legend()

p = Point(individual_cloud.lon.data, individual_cloud.lat.data)
circle = p.buffer(1)
x, y = zip(*list(circle.exterior.coords))

ax.plot(
    x,
    y,
    zorder=20,
    color="k",
)

In [ ]:
fig, axs = plt.subplots(ncols=2)
plot_psd_scatter(fig=fig, ax=axs[0], ds_cloudcomposite=cc_individual, split_radius=4.5e-5)
plot_msd_scatter(fig=fig, ax=axs[1], ds_cloudcomposite=cc_individual, split_radius=4.5e-5)

N_total = cc_individual["particle_size_distribution"].sum().values
N_mean = cc_individual["particle_size_distribution"].mean().values

M_total = cc_individual["mass_size_distribution"].sum().values * 1e3
M_mean = cc_individual["mass_size_distribution"].mean().values * 1e3

axs[0].set_title(
    "Particle size distribution\n"
    + r"$N_{total}= $"
    + f"{N_total:.2e}"
    + r"  $N_{mean}= $"
    + f"{N_mean:.2e}",
    fontsize=10,
)
axs[1].set_title(
    "Mass size distribution\n"
    + r"$M_{total}= $"
    + f"{M_total:.3f}"
    + r"$g m^{-3}$"
    + r"  $M_{mean}= $"
    + f"{M_mean:.2e}"
    + r"$g m^{-3}$",
    fontsize=10,
)

fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(20, 10))

subfigs = fig.subfigures(1, 2, wspace=0.07)

plot_thermodynamics(subfigs[0], drop_sondes, individual_cloud)
plot_thermodynamics(subfigs[1], ds_individual, individual_cloud)